<a href="https://colab.research.google.com/github/miladrezae/Machine-learning/blob/main/TensorFlow_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


In [1]:
import matplotlib as plt
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras

builder=tfds.builder('rock_paper_scissors')
info=builder.info

ds_train =tfds.load(name='rock_paper_scissors',split='train') #It's in the info so we can call it
ds_test=tfds.load(name='rock_paper_scissors',split='test')


train_images=[example['image'] for example in ds_train] #example['image'] is how you access tfds dataset

train_images=np.array([example['image'].numpy()[:,:,0] for example in ds_train]) #We only picked RED at [:,:,0]
train_labels=np.array([example['label'].numpy() for example in ds_train])
test_images=np.array([example['image'].numpy()[:,:,0] for example in ds_test])
test_labels=np.array([example['label'].numpy() for example in ds_test])

train_images=train_images.reshape(2520, 300, 300,1) #from (2520, 300, 300)
test_images=test_images.reshape(372, 300, 300, 1)
train_images=train_images.astype('float32')
test_images=test_images.astype('float32')

train_images/=255   #This scaled every value between 0 and 1
test_images/=255

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incomplete0E5CS1/rock_paper_scissors-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incomplete0E5CS1/rock_paper_scissors-test.tfrecord


Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


In [2]:
model=keras.Sequential([
    keras.layers.Conv2D(64,3,activation='relu',input_shape=(300,300,1)), #filters, kernel_size(window size), strides(movement of window by default is 1)
    keras.layers.Conv2D(32,3,activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(3,activation='softmax')
])

model.compile(optimizer='adam',
             loss=keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])

model.fit(train_images,train_labels,epochs=5,batch_size=32)

Epoch 1/5
79/79 [==============================] - 22s 174ms/step - loss: 8.7318 - accuracy: 0.5941
Epoch 2/5
79/79 [==============================] - 14s 171ms/step - loss: 0.2030 - accuracy: 0.9576
Epoch 3/5
79/79 [==============================] - 14s 175ms/step - loss: 0.0394 - accuracy: 0.9928
Epoch 4/5
79/79 [==============================] - 14s 172ms/step - loss: 0.0110 - accuracy: 0.9967
Epoch 5/5
79/79 [==============================] - 13s 168ms/step - loss: 0.0094 - accuracy: 0.9988


In [13]:
model=keras.Sequential([
    keras.layers.AveragePooling2D(6,3,input_shape=(300,300,1)),#We make 6 by6 pixels and we move it by 3, making it 100x100
    keras.layers.Conv2D(64,3,activation='relu'), #filters, kernel_size(window size), strides(movement of window by default is 1)
    keras.layers.Conv2D(32,3,activation='relu'),
    keras.layers.MaxPool2D(2,2), #Smaller parameter size
    keras.layers.Dropout(0.5), #0.5 in research is a good common rate
    keras.layers.Flatten(),
    # keras.layers.Dense(128,activation='relu'), #another layer before we drop it to 3
    keras.layers.Dense(3,activation='softmax')
])

model.compile(optimizer='adam',
             loss=keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])

model.fit(train_images,train_labels,epochs=5,batch_size=32)

Epoch 1/5
79/79 [==============================] - 2s 26ms/step - loss: 1.2494 - accuracy: 0.5157
Epoch 2/5
79/79 [==============================] - 2s 25ms/step - loss: 0.4076 - accuracy: 0.8951
Epoch 3/5
79/79 [==============================] - 2s 25ms/step - loss: 0.1594 - accuracy: 0.9675
Epoch 4/5
79/79 [==============================] - 2s 25ms/step - loss: 0.0909 - accuracy: 0.9809
Epoch 5/5
79/79 [==============================] - 2s 25ms/step - loss: 0.0611 - accuracy: 0.9856


In [14]:
model.evaluate(test_images,test_labels)

12/12 [==============================] - 0s 12ms/step - loss: 0.9220 - accuracy: 0.6855


[0.9219591021537781, 0.6854838728904724]

In [45]:
# pip install -U keras-tuner
from kerastuner.tuners import RandomSearch
def build_model(hp):
    model=keras.Sequential()
    model.add(keras.layers.AveragePooling2D(6,3, input_shape=(300,300,1)))
    for i in range(hp.Int('Conv Layers',min_value=0,max_value=3)):
      model.add(keras.layers.Conv2D(hp.Choice(f"layer_{i}_filters",[16,32,64]),3,activation='relu'))

    
    model.add(keras.layers.MaxPool2D(2,2))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Flatten())
    
    model.add(keras.layers.Dense(hp.Choice('Dense layer',[64,128,256,1024]),activation='relu'))
    
    model.add(keras.layers.Dense(3, activation='softmax'))
    
    model.compile(optimizer='adam',
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])
    
    return model

tuner=RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=32,
    directory='./new_directory'

)

tuner.search(train_images,train_labels, validation_data=(test_images,test_labels),epochs=10,batch_size=32)



Trial 32 Complete [00h 00m 19s]
val_accuracy: 0.7795698642730713

Best val_accuracy So Far: 0.7903226017951965
Total elapsed time: 00h 11m 12s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [46]:
best_model=tuner.get_best_models()[0]
best_model.evaluate(test_images,test_labels)

12/12 [==============================] - 0s 11ms/step - loss: 0.5672 - accuracy: 0.8075


[0.5892497897148132, 0.7903226017951965]

In [39]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
average_pooling2d (AveragePo (None, 99, 99, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 97, 97, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 95, 95, 32)        18464     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 47, 47, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 47, 47, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 70688)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              7

In [40]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
Dense layer: 1024
Score: 0.7446236610412598
Trial summary
Hyperparameters:
Dense layer: 128
Score: 0.6881720423698425
Trial summary
Hyperparameters:
Dense layer: 64
Score: 0.6827957034111023
Trial summary
Hyperparameters:
Dense layer: 256
Score: 0.6774193644523621
